### Time Estimate


In [1]:
CONFIG = "./configs/random_10M_2024"
import json
args = None
with open(CONFIG) as f:
    args = json.load(f)
    print(args)
args["gradients_per_file"] = 10000

{'dataset_folder': './curricula/datasets/curriculum_10M_2024', 'raw_dataset_folder_babylm': './train_10M', 'curriculum_path': './curricula/random_10M_2024', 'eval_dataset_folder': './curricula/datasets/curriculum_10M_2024_eval', 'epochs': 10}


In [2]:
total_in_h = 0


In [3]:
from datasets import load_dataset
import datasets
t = lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, max_length=512)

dataset = datasets.load_from_disk(args["dataset_folder"])
dataset.set_transform(t)





In [4]:
print(len(dataset) / args["gradients_per_file"], "chunks per checkpoint")

117.9014 chunks per checkpoint


#### gradient extraction

In [5]:
args["num_processes"] = 6
TIME_PER_BATCH_IN_MIN = 441/60

In [6]:
time_if = ((((len(dataset) / args["gradients_per_file"]) *args["epochs"]) /args["num_processes"]) * TIME_PER_BATCH_IN_MIN)/60
print(time_if, "hours total", time_if/args["epochs"], "hours per checkpoint")
total_in_h+=time_if

24.07153583333333 hours total 2.407153583333333 hours per checkpoint


In [7]:
SIZE_IN_GB_OF_CHUNK = 7.4


In [8]:
print((len(dataset) / args["gradients_per_file"]) * SIZE_IN_GB_OF_CHUNK/1000, "TB disk space required")

0.87247036 TB disk space required


#### mean dot

In [9]:
args["batch_size"] = 20
args["num_processes"] = 4

SIZE_IN_GB_OF_CHUNK = 7.4 # ls
time_single_chunk = 15.36 # get from log
time_load_task = 110 # get from log

In [10]:
print("RAM requirements", (args["batch_size"]+1)*args["num_processes"]*SIZE_IN_GB_OF_CHUNK, "GB")

RAM requirements 621.6 GB


In [11]:


num_chunks =(((len(dataset) / args["gradients_per_file"])))
num_batches = num_chunks / args["batch_size"]

time_single_batch = (num_chunks * time_single_chunk) + time_load_task
time_single_epoch = (num_batches * time_single_batch) 
print("Time for a single epoch and one process", time_single_epoch /60/60, "hours")
total_merge = (time_single_epoch * args["epochs"] /args["num_processes"])/60/60
print("Total time with multiprocessing", total_merge, "hours")
total_in_h += total_merge

Time for a single epoch and one process 3.1456183649070217 hours
Total time with multiprocessing 7.864045912267555 hours


## Total

In [12]:
print(total_in_h, "hours")

31.935581745600885 hours
